In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:


# Set up Selenium headless browser
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

data = []
def scrape_page(year):
    if year == 2024:
        return
    url = "https://eurovisionworld.com/eurovision/" + str(year)
    driver.get(url)
    time.sleep(3)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Find the main winners table
    table = soup.find("table", class_="v_table")

    # Loop through each row, skip malformed or empty ones
    for row in table.find_all("tr"):
        cols = row.find_all("td")
        
        # Skip rows that don't have enough data
        if len(cols) < 4:
            continue

        # Parse year and skip 2020 (canceled year)
        

        country = cols[1].get_text(strip=True)

        # Handle song and artist (may be combined or malformed)
        song_artist_cell = cols[2]
        song = ""
        artist = ""

        # Some song cells are split into text + <span class="v_artist"> tag
        if song_artist_cell:
            parts = song_artist_cell.get_text(separator="|", strip=True).split("|")
            if len(parts) >= 2:
                song = parts[0]
                artist = parts[1]
            elif len(parts) == 1:
                song = parts[0]

        points = cols[3].get_text(strip=True)

        data.append([year, country, song, artist, points])
        print([year, country, song, artist, points])

for year in range(2024, 1955, -1):
    scrape_page(year)

driver.quit()
# Create DataFrame
df = pd.DataFrame(data, columns=["Year", "Country", "Song", "Artist", "Points"])
print(df.head())

# # Save to CSV
# df.to_csv("all_songs.csv", index=False)


MaxRetryError: HTTPConnectionPool(host='localhost', port=56819): Max retries exceeded with url: /session/e6ded1084236b9aff462ba937d05000f/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000202818696D0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))